# Import

In [1]:
import torch
import torchvision

from glcic.networks.completion_network import CompletionNetwork
from glcic.networks.discriminators import Discriminator
from glcic.trainers.conjugate_trainer import train
from glcic.utils import *

# Prepare

In [2]:
# load the dataset
train_dataset_dir = '../data/train/'
train_dataset = torchvision.datasets.ImageFolder(root=train_dataset_dir, transform=torchvision.transforms.ToTensor())
train_dataloader = get_dataloader(train_dataset_dir, train_dataset_dir+"barn/00001008.jpg", batch_size=2)

In [3]:
# others
cn = CompletionNetwork().cuda()
cn_optimizer = torch.optim.Adadelta(cn.parameters())

discriminator = Discriminator().cuda()
discriminator_optimizer = torch.optim.Adadelta(discriminator.parameters())

num_batch = 10
replacement_val = torch.tensor([0.5062325495504219, 0.4255871700324652, 0.38299278586700136])

# Test

In [4]:
# conjugate training
train(
    cn,
    discriminator,
    cn_optimizer,
    discriminator_optimizer,
    train_dataloader,
    num_batch,
    replacement_val,
    )


### BATCH 1/10 ###


AssertionError: 